# Data Collecting
Author: ZHANG Yun

In this section, we collect the commodities' price data, recent news, and some social media data.

## Load Packages

In [14]:
!pip install investpy
!pip3 install --upgrade -e git+https://github.com/twintproject/twint.git@origin/master#egg=twint

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
^C


In [20]:
# API
import investpy
import investpy.commodities as invc
# Crawlers
import requests
from bs4 import BeautifulSoup
# import twint
# import nest_asyncio

# Others
import os
import queue
import json
import pandas as pd
import numpy as np
import concurrent.futures
import time

## Collect Commodities Price Data

The `investpy` package is used to collect commodities price data from [investing.com](https://www.investing.com/commodities/grains).

Usage of this package can be referred from https://investpy.readthedocs.io/_api/commodities.html?highlight=commodity

In [21]:
# Commodities that we are interested in
COMMODITY_LIST = ['US Corn', 'US Soybeans', 'US Wheat', 'Oats', 'Rough Rice']
FROM_DATE = "01/11/2010"
TO_DATE =  "01/11/2020"

In [22]:
# Overview of the real time data for the commodities in our list
overview = invc.get_commodities_overview('grains')
overview[overview['name'].isin(COMMODITY_LIST)]

,country,name,last,last_close,high,low,change,change_percentage,currency
1,united states,Rough Rice,12.51,12.555,12.523,12.50,0.000,0.00%,USD
4,united states,US Soybeans,1143.62,1162.000,1163.750,1143.00,-18.38,-1.58%,USD
5,united states,US Wheat,569.50,565.000,575.380,564.75,+4.50,+0.80%,USD
6,united states,US Corn,410.50,414.000,415.250,410.25,-3.50,-0.85%,USD
7,united states,Oats,290.20,291.800,291.600,289.50,-1.60,-0.55%,USD


In [23]:
commodities_info = pd.concat([invc.get_commodity_information(com) for com in COMMODITY_LIST])
commodities_info

,Commodity Name,Prev. Close,Month,Tick Size,Open,Contract Size,Tick Value,Todays Range,Settlement Type,Base Symbol,52 wk Range,Settlement Day,Point Value,1-Year Change,Last Rollover Day,Months
0,US Corn,414.000,Dec 20,0.0025,414.000,NaN,12.5,410.25 - 415.25,Physical,ZC (C),300.25 - 430.5,30/11/2020,$50,10.71%,09/09/2020,NaN
0,US Soybeans,1162.000,Jan 21,0.0025,1163.250,NaN,12.5,"1,143 - 1,163.75",Physical,ZS (S),"808.38 - 1,199.62",14/01/2021,$50,32.74%,03/11/2020,NaN
0,US Wheat,565.000,Dec 20,0.0025,575.380,NaN,12.5,564.75 - 575.38,Physical,ZW (W),468.25 - 637.62,30/11/2020,$50,5.16%,01/09/2020,NaN
0,Oats,291.800,None,0.0025,291.400,NaN,12.5,289.5 - 291.6,Physical,None,248.25 - 350,None,$50,- 10.47%,None,NaN
0,Rough Rice,12.555,Jan 21,0.0050,12.508,"2,000 cwt",10.0,12.5 - 12.523,Physical,ZR (RR),11.215 - 23.565,31/12/2020,$20,0.32%,25/10/2020,NaN


In [31]:
# Collect the commodities OLHC data and store them in CSV file
file_path = os.getcwd()
if os.path.exists(file_path + "\data\\") == False:
    os.mkdir("data")
for commodity in COMMODITY_LIST:
    name = commodity.lower()
    invc.get_commodity_historical_data(commodity, FROM_DATE, TO_DATE).to_csv(f'./data/{name}_historical.csv')

## Collect Commodity News
Here we adopt a [multi-thread method](https://beckernick.github.io/faster-web-scraping-python/) to fastly scrape data from web. In case there are duplicated links for news or analysis, we store the links in a set.

In [7]:
# Variables define
BASE_URL = 'https://www.investing.com'
SUFFIX = ['-news', '-opinion']
ROOT_URLS = {com:[BASE_URL+'/commodities/'+com.lower().replace(' ', '-')+suf for suf in SUFFIX] for com in COMMODITY_LIST}
MAX_PAGES = 10
MAX_THREADS = 30
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"}#,
          #"Cookie": '''APID=UP8f2f9e6c-2b07-11eb-a7c2-023a5f2e6620; unique_ad_source_impression="1243603%2C1258578__TIME__2020-11-20+01%3A23%3A25"; B=dfl7sg1freubi&b=3&s=je; BTS=1605864208; adaptv_unique_user_cookie="1531969023908454971__TIME__2020-11-20+01%3A23%3A47"'''}
urls = {}

In [ ]:
# Get links of news and analysis from investing.com
# If the links are already collected, skip this and next cell
for com in COMMODITY_LIST:
  urls[com] = []
  for url in ROOT_URLS.get(com):
    for page in range(1, MAX_PAGES+1):
      resp = requests.get(f"{url}/{page}", headers=header)
      if resp.status_code != 200:
        print(f"Abnormal status code {resp.status_code} returned from {url}/{page}")
        # break
        continue
      soup = BeautifulSoup(resp.content, "html.parser")
      news = soup.find_all("a", attrs={"class":"title"})
      urls[com] += [BASE_URL + x["href"] for x in news]
    time.sleep(0.25)

In [17]:
# Save the collected links into file url.json
print(f"{sum(list(map(len, urls)))} pieces of news and analysis found.")
with open('urls.json', 'w') as f:
  f.write(json.dumps(urls))

40 pieces of news and analysis found.


In [67]:
# Load the file containing links of news and analysis
with open("urls.json", 'r') as f:
    urls = f.read()
urls = json.loads(urls)

News data are collectoed by WebScraper, a selenium based Chrome extension. Create the sitemap by following configuration.

```
{"_id":"investing_news","startUrl":["https://www.investing.com/commodities/us-corn-news/[1-10]","https://www.investing.com/commodities/us-corn-opinion/[1-10]","https://www.investing.com/commodities/us-soybeans-news/[1-10]","https://www.investing.com/commodities/us-soybeans-opinion/[1-10]","https://www.investing.com/commodities/us-wheat-news/[1-10]"],"selectors":[{"id":"news_one","type":"SelectorLink","parentSelectors":["_root"],"selector":"#leftColumn a.title","multiple":true,"delay":0},{"id":"detail","type":"SelectorText","parentSelectors":["news_one"],"selector":"div.contentSectionDetails","multiple":false,"regex":"","delay":0},{"id":"article","type":"SelectorHTML","parentSelectors":["news_one"],"selector":"div.WYSIWYG.articlePage","multiple":false,"regex":"","delay":0}]}
```

Our cralwer cannot beat the anti-crawler mechanism of investing.com.

In [84]:
def download_url(url):
    res = requests.get(url, headers=header)
    soup = BeautifulSoup(res.text, 'html.parser')
    title = soup.title.text
    date = soup.select_one("div.contentSectionDetails > span").text
    content = " ".join([p.text for p in soup.find_all("p", {"dir":"ltr"})])
    js = {"url": url, "title": title, "date": date, "content": content}
    
    print(title)
    print(content[:100])
    print("=============================================")
    
    with open("corpus\{}.json".format(title), "wb") as f:
        f.write(json.dumps(js))

    time.sleep(0.25)
  
def download_news(urls):
    file_path = os.getcwd()
    if os.path.exists(file_path + "\corpus") == False:
      os.mkdir("corpus")
    threads = min(MAX_THREADS, len(urls))
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(download_url, urls)

In [88]:
# url = "https://www.investing.com/analysis/silver-the-poor-mans-gold-delivers-richest-performance-in-40-years-in-july-200532554"
url = urls['US Corn'][0]
print(url)
res = requests.get(url, headers=header)
soup = BeautifulSoup(res.text, 'html.parser')
date = soup.select_one("div.contentSectionDetails > span").text
title = soup.title.text
content = " ".join([p.text for p in soup.find_all("p", {"dir":"ltr"})])   
js = {"url": url, "title": title, "date": date, "content": content}
with open("test.json", 'w') as f:
    f.write(json.dumps(js))
    
# with open("corpus\{}".format(title), "wb") as fh:
#     fh.write(res.content)

https://www.investing.com/news/stock-market-news/sp-closes-at-record-as-tech-shows-strength-2357468


In [62]:
# soup.select("p")
# " ".join([p.text for p in soup.find_all("p", {"dir":"ltr"})])
soup.select_one("div.contentSectionDetails > span").text

'Jul 31, 2020 04:49AM ET'

In [85]:
# for ulist in urls.values():
#     download_news(ulist)
download_news(urls['US Corn'])

China Hits U.S. Where It Hurts With 25% Tariff on Soybeans By Bloomberg

World stocks advance as trade war worries ease By ReutersGrain glut leaves U.S. farmers facing losses from specialty corn By Reuters


Soybeans May Be Next Market to Surge as U.S. Showers Drag On By Bloomberg


Stocks - Boeing, Walgreens Boost Dow; China Stalls Market By Investing.comHog, Corn Futures Jump as U.S., Mexico Agree on Trade Deal By Bloomberg

China Tariffs Blindside U.S. Farmers at Worst Possible Time By Bloomberg



What to Watch in Commodities: Winners and Losers in Second Half By Bloomberg

Top 5 Things to Know in the Market on Wednesday By Investing.comS&P Closes at Record as Tech Shows Strength By Investing.com

Strong dollar impeding U.S. grain export growth By Investing.comDollar Weakens; Stimulus Expected to Combat Pandemic By Investing.com



Tesla Becomes Market Cap Behemoth Ahead of S&P 500 Listing By Investing.com
By Peter Nurse Investing.com - The dollar weakened in early European trade F

## Collect data from social media

In [ ]:
nest_asyncio.apply()
key_words = ["commodity", "wheat", "corn", "soybean"]

In [ ]:
def tweet_search(grain, verified = False, min_likes = 2):
  # dates = ["2010-01-01", "2012-01-01", "2014-01-01", "2016-01-01", "2018-01-01", "2020-12-01"]
    c = twint.Config()
    c.Limit = 50000
    c.Search = ["commodity", grain] # keywords 
    c.Min_likes = min_likes # filter out the tweet with minimal likes/replies/retweets
    c.verified = verified
    # c.Pandas = True
    c.lowercase = True
    c.Store_csv = True
    c.Hide_output = True
    c.Stats = True  
    c.Output = f"{grain}_tweet.csv"
    # for i in range(5):
    #   since = dates[i]
    #   until = dates[i+1]
    #   c.Since = since # restrict the time
    #   c.Until = until
    #   c.Output = f"{grain}_tweet_{i}.csv"
    c.Since = "2010-01-01"
    c.Until = "2020-12-01"
    # Run
    twint.run.Search(c)
    # tweet_df = twint.storage.panda.Tweets_df

In [ ]:
for grain in key_words:
    tweet_search(grain, min_likes=0)
    print(f"{grain} done!")